This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 20; Grid Count = 20; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_OProbStudy2 }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-03-29 09:28:28Z

In [18]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:06:32 PM	20160aa8...
#1: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:06:14 PM	627ed621...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:05:51 PM	ecfdef17...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 11:37:07 AM	ce641304...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 11:37:31 AM	ea004fc4...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 11:35:34 AM	2384c407...
#6: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 12:07:37 PM	8155bf06...
#7: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 12:07:15 PM	823ebbc6...
#8: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 11:35:57 AM	7090bb5d...
#9: XDGBowShock_TwoLs	XDGBS-

In [9]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

20160aa8-211b-401e-a3e6-ecd886f50ca8

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$


For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock presented, the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=4$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 4\sqrt{\gamma}$$

In [10]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [11]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [22]:
var sessions = database.Sessions.Where(s => s.SuccessfulTermination); //plot all 
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:06:32 PM	20160aa8...
#1: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:06:14 PM	627ed621...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:05:51 PM	ecfdef17...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 11:37:07 AM	ce641304...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 11:37:31 AM	ea004fc4...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 11:35:34 AM	2384c407...
#6: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 12:07:37 PM	8155bf06...
#7: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 12:07:15 PM	823ebbc6...
#8: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	3/29/2024 11:35:57 AM	7090bb5d...
#9: XDGBowShock_TwoLs	XDGBS-

In [26]:
sessions=sessions.Where(s => (s.Name.Contains("iFlx0") &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:06:32 PM	20160aa8...
#1: XDGBowShock_TwoLs	XDGBS-p3-15x48-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:06:14 PM	627ed621...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 12:05:51 PM	ecfdef17...
#3: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	3/29/2024 11:35:34 AM	2384c407...


# A Helper Function to obtain different plot formats

In [27]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Etnhalpy Error Plot


In [28]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M511.2,57.1 L564.6,57.1 M45.6,132.3 L50.0,136.1 L54.4,139.0 L58.9,141.6 L63.3,146.6 L67.7,155.6
 L72.1,169.3 L76.5,173.0 L80.9,175.4 L85.4,178.8 L89.8,180.7 L94.2,181.3 L98.6,181.9 L103.0,182.2
 L107.5,182.4 L111.9,182.3 L116.3,182.2 L120.7,182.3 L125.1,182.2 L129.5,182.2 L134.0,182.2 L138.4,182.2
 L142.8,182.2 L147.2,182.2 L151.6,182.2 L156.0,182.2 L160.5,182.2 L164.9,182.2 L169.3,182.2 L173.7,182.2
 L178.1,182.2 L182.6,182.2 L187.0,182.2 L191.4,182.2 L195.8,182.2 L200.2,182.2 L204.6,182.2 L209.1,182.2
 L213.5,182.2 L217.9,182.2 L222.3,182.2 L226.7,182.2 L231.2,182.2 L235.6,182.2 L240.0,182.2 L244.4,182.2
 L248.8,182.2 L253.2,182.2 L257.7,182.2 L262.1,182.2 L266.5,182.2 L270.9,182.2 L275.3,182.2 L279.8,182.2
 L284.2,182.2 L288.6,182.2 L293.0,182.2 L297.4,182.2 L301.8,182.2 L306.3,182.2 L310.7,182.2 L315.1,182.2
 L319.5,182.2 L323.9,182.2 L328.4,182.2 L332.8,182.2 L337.2,182.2 L341.6,182.2 L346.0,182.2 L350.4,182.2
 L354.9,182.2 L359.3,182.2 L363.7,182.2 L368.1,182.2 L372.5,182.2 L376.9,182.2 L381.4,182.2 L385.8,182.2
 L390.2,182.2 L394.6,182.2 L399.0,247.2 L403.5,277.4 L407.9,333.3 L412.3,374.8 L416.7,375.4 L421.1,375.6
 L425.5,375.5 L430.0,375.4 L434.4,375.5 L438.8,375.5 L443.2,375.5 L447.6,375.5 L452.1,375.5 L456.5,375.5
 L460.9,375.5 L465.3,375.5 L469.7,375.5 L474.1,375.5 L478.6,375.5 L483.0,375.5 L487.4,375.5 L491.8,375.5
 L496.2,375.5 L500.7,375.5 L505.1,375.5 L509.5,375.5 L513.9,375.5 L518.3,375.5 L522.7,375.5 L527.2,375.5
 L531.6,375.5 L536.0,375.5 L540.4,375.5 L544.8,375.5 L549.3,375.5 L553.7,375.5 L558.1,375.5 L562.5,375.5
 L566.9,375.5 L571.3,375.5 L575.8,375.5 L580.2,375.5 L584.6,375.5 L589.0,375.5 L593.4,375.5 L597.8,375.5
 L602.3,375.5 L606.7,375.5 L611.1,375.5 L615.5,375.5 L619.9,375.5 L624.4,375.5 L628.8,375.5 L633.2,375.5
 L637.6,375.5 L642.0,375.5 L646.4,375.5 L650.9,375.5 L655.3,375.5 L659.7,375.5 L664.1,375.5 L668.5,375.5
 L673.0,375.5 L677.4,375.5 L681.8,375.5 L686.2,375.5 L690.6,375.5 L695.0,375.5 L699.5,463.6 L703.9,472.2
 L708.3,472.4 L712.7,472.4 L717.1,472.3 L721.6,472.4 L726.0,472.4 L730.4,472.4 L734.8,472.4 L739.2,472.4
 L743.6,472.4 L748.1,472.4 L752.5,472.4 L756.9,472.4 L761.3,472.4 L765.7,472.4 L770.2,472.4 L774.6,472.4
 L779.0,472.4 L783.4,472.4 L787.8,472.4 L792.2,472.4 L796.7,472.4 L801.1,472.4 L805.5,472.4 L809.9,472.4
 L814.3,472.4 L818.7,472.4 L823.2,472.4 L827.6,472.4 L832.0,472.4 L836.4,472.4 L840.8,472.4 L845.3,472.4
 L849.7,472.4 L854.1,472.4 L858.5,472.4 L862.9,472.4 L867.3,472.4 L871.8,472.4 L876.2,472.4 L880.6,472.4
 L885.0,472.4 L889.4,472.4 L893.9,472.4 L898.3,472.4 L902.7,472.4 L907.1,472.4 L911.5,472.4 L915.9,472.4
 L920.4,472.4 L924.8,472.4 L929.2,472.4 L933.6,541.1 L938.0,540.6 L942.5,540.6 L946.9,540.6 L951.3,540.7
 L955.7,540.8 L960.1,540.8 L964.5,540.7 L969.0,540.7 L973.4,540.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-15x48-agg0.4-iPrb0-iFlx

## Residual History

In [30]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M22.4,57.1 L75.8,57.1 M45.6,92.6 L48.4,96.2 L51.2,98.0 L54.1,99.5 L56.9,99.7 L59.7,101.5
 L62.5,113.6 L65.3,131.1 L68.1,134.9 L71.0,144.2 L73.8,151.1 L76.6,175.6 L79.4,177.3 L82.2,225.9
 L85.1,231.9 L87.9,208.8 L90.7,235.6 L93.5,240.9 L96.3,248.2 L99.1,259.6 L102.0,267.3 L104.8,272.1
 L107.6,275.9 L110.4,279.4 L113.2,287.8 L116.0,289.2 L118.9,289.3 L121.7,289.7 L124.5,295.2 L127.3,295.3
 L130.1,299.1 L133.0,300.3 L135.8,301.7 L138.6,302.0 L141.4,302.0 L144.2,302.0 L147.0,302.5 L149.9,302.5
 L152.7,302.6 L155.5,303.4 L158.3,304.1 L161.1,304.8 L164.0,309.6 L166.8,309.3 L169.6,309.7 L172.4,311.4
 L175.2,311.6 L178.0,311.7 L180.9,311.8 L183.7,311.8 L186.5,311.8 L189.3,311.8 L192.1,311.8 L195.0,312.3
 L197.8,312.7 L200.6,312.8 L203.4,312.9 L206.2,313.0 L209.0,313.0 L211.9,313.8 L214.7,313.6 L217.5,314.0
 L220.3,314.0 L223.1,314.0 L226.0,314.1 L228.8,314.2 L231.6,314.3 L234.4,314.3 L237.2,314.3 L240.0,314.5
 L242.9,314.4 L245.7,314.4 L248.5,314.6 L251.3,314.7 L254.1,314.7 L256.9,314.8 L259.8,314.8 L262.6,314.8
 L265.4,314.5 L268.2,314.8 L271.0,126.7 L273.9,139.1 L276.7,184.1 L279.5,263.5 L282.3,288.2 L285.1,293.6
 L287.9,306.6 L290.8,309.6 L293.6,313.4 L296.4,322.9 L299.2,326.1 L302.0,332.4 L304.9,333.5 L307.7,336.4
 L310.5,337.3 L313.3,340.0 L316.1,341.1 L318.9,342.7 L321.8,344.2 L324.6,344.4 L327.4,345.6 L330.2,346.8
 L333.0,347.5 L335.9,349.1 L338.7,350.0 L341.5,350.1 L344.3,350.9 L347.1,351.2 L349.9,353.4 L352.8,353.8
 L355.6,354.4 L358.4,354.6 L361.2,355.2 L364.0,355.2 L366.9,356.1 L369.7,356.6 L372.5,356.6 L375.3,356.9
 L378.1,357.2 L380.9,357.4 L383.8,357.7 L386.6,358.0 L389.4,358.1 L392.2,358.4 L395.0,358.4 L397.8,358.6
 L400.7,358.7 L403.5,358.8 L406.3,358.8 L409.1,358.9 L411.9,359.0 L414.8,359.2 L417.6,359.2 L420.4,359.3
 L423.2,359.4 L426.0,359.6 L428.8,359.6 L431.7,359.7 L434.5,359.8 L437.3,359.8 L440.1,360.0 L442.9,360.0
 L445.8,360.0 L448.6,360.1 L451.4,360.1 L454.2,360.1 L457.0,360.1 L459.8,360.1 L462.7,273.7 L465.5,264.4
 L468.3,351.7 L471.1,362.6 L473.9,367.2 L476.8,376.6 L479.6,386.2 L482.4,394.0 L485.2,398.5 L488.0,402.6
 L490.8,404.1 L493.7,405.7 L496.5,408.3 L499.3,409.7 L502.1,410.4 L504.9,411.1 L507.8,412.7 L510.6,412.8
 L513.4,413.3 L516.2,413.9 L519.0,414.3 L521.8,414.8 L524.7,414.8 L527.5,414.9 L530.3,415.5 L533.1,415.9
 L535.9,416.0 L538.7,416.6 L541.6,417.7 L544.4,417.9 L547.2,418.1 L550.0,418.3 L552.8,419.0 L555.7,419.1
 L558.5,419.5 L561.3,419.8 L564.1,420.0 L566.9,420.1 L569.7,420.1 L572.6,420.3 L575.4,420.4 L578.2,420.4
 L581.0,420.9 L583.8,421.0 L586.7,421.1 L589.5,421.2 L592.3,421.4 L595.1,421.6 L597.9,421.7 L600.7,421.8
 L603.6,422.0 L606.4,422.0 L609.2,422.0 L612.0,329.5 L614.8,348.1 L617.7,352.5 L620.5,375.1 L623.3,379.3
 L626.1,394.0 L628.9,403.4 L631.7,409.3 L634.6,412.4 L637.4,413.8 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

save results to text file

In [50]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [22]:
var ses= sessions.Where(si => si.Name.Contains("iFlux0"));
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:43:47 AM	ff833be6...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:24 AM	041bb15f...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo*	8/25/2023 10:44:42 AM	1ae3733e...


Number of cells

In [23]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280, 5120 ]

helper arrays

In [24]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(ses.Count(),4);


Function used to calculate the EOCs

In [25]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

In [32]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.WriteLine($"P={p},sess={iSess}");
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
        }
    }
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    count++;
}
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

P=0,sess=0
P=0,sess=1
P=0,sess=2
P=0,sess=3
P=1,sess=0
P=1,sess=1
P=1,sess=2
P=1,sess=3
P=2,sess=0
P=2,sess=1
P=2,sess=2
P=2,sess=3
P=3,sess=0
P=3,sess=1
P=3,sess=2
P=3,sess=3
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3

In [33]:
EOCs

[ 0.9038108734340952, 1.921975745497684, 2.3656730498000016, 2.680293477108991 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [28]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlux0"));
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...


In [29]:
var si = sessions.Pick(0);
si

XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iProb0-iFlux0-FphiType0-NewAgglo	8/25/2023 10:44:02 AM	16765961...

plot it

In [59]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=5;
var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
foreach(var timestep in si.Timesteps){
    if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
            IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
            var SF = lts.GetShadowFields();
            var texplot = new Tecplot(SF[0].GridDat, 2);
            texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
    }
}